<h1> Node Classification using Graph Convolutional Networks </h1>

This node classification task uses CORA dataset from https://linqs.soe.ucsc.edu/data<br>
The dataset consists of <b>2708</b> nodes which correspond to scientific publications.<br>
The nodes are classified into <b>7</b> categories indicating the topics of each document.<br>
The edges indicate whether a document is cited by the other or vice versa.<br>
Each node has <b>1433</b> features which is described by a 0/1-valued vector, indicating the bag-of-words from the dictionary.<br>

<br> This is an undirected graph problem

In [1]:
#importing dependencies

import numpy as np
import os
import networkx as nx
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

from spektral.layers import GraphConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

from collections import Counter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

<h2> Data Loading and Preprocessing </h2><br>
We are going to use the edges connecting the  (from file <b>cora.cites</b>).<br>
The nodes are loaded from file <b>cora.content</b>.

In [2]:
#loading the data

all_data = []
all_edges = []

for root,dirs,files in os.walk('./cora'):
    print(root,' ',dirs,' ',files)
    for file in files:
        #print(file)
        if '.content' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines()) #splits the files by new line(\n)
        elif 'cites' in file:
            with open(os.path.join(root,file),'r') as f:
                all_edges.extend(f.read().splitlines())

                
#Shuffle the data because the raw data is ordered based on the label
random_state = 77
all_data = shuffle(all_data,random_state=random_state)

./cora   []   ['cora.cites', 'cora.content', 'cora.edges', 'cora.node_labels', 'README', 'readme.html']


In [3]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [4]:
print(len(all_data))
print(len(all_edges))
type(all_edges[0])
print((all_data[110]))

2708
5429
504	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	

In <b>cora.content</b> file:<br>
The <b>first</b> element indicates the <b>node name</b><br>
The <b>second</b> until the last second elements indicate the <b>node features</b><br>
The <b>last</b> element indicates the <b>label of that particular node</b><br>

In <b>cora.cites</b> file:<br>
Each line indicates the tuple of connected nodes


<h3>Parsing the data</h3>

In [5]:
#parse the data
labels = []
nodes = []
X = []

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(elements[-1])
    X.append(elements[1:-1])
    nodes.append(elements[0])
#print(elements)
X = np.array(X,dtype=int) #node feature matrix??
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features
print('X shape: ', X.shape)


#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))# string to tuple conversion

print('\nNumber of nodes (N): ', N)
print('Number of features (F) of each node: ', F)
print('\nCategories: ', set(labels)) #set gives the unique labels

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)

X shape:  (2708, 1433)

Number of nodes (N):  2708
Number of features (F) of each node:  1433

Categories:  {'Theory', 'Neural_Networks', 'Reinforcement_Learning', 'Rule_Learning', 'Genetic_Algorithms', 'Case_Based', 'Probabilistic_Methods'}

Number of classes:  7


<h3> Select examples for training, validation, and test then set the mask</h3>

In [6]:
def limit_data(labels,limit=20,val_num=500,test_num=1000):
    '''
    Get the index of train, validation, and test data
    '''
    label_counter = dict((l, 0) for l in labels)
    print('label_counter: ',label_counter)
    train_idx = []

    for i in range(len(labels)):
        label = labels[i]
        if label_counter[label]<limit:
            #add the example to the training data
            train_idx.append(i)
            label_counter[label]+=1
        
        #exit the loop once we found 20 examples for each class
        if all(count == limit for count in label_counter.values()):
            break
    
    #get the indices that do not go to traning data
    rest_idx = [x for x in range(len(labels)) if x not in train_idx]
    print(len(labels))
    print((rest_idx[:10]))
    #get the first val_num
    val_idx = rest_idx[:val_num] # from 0 to val_num-1
    test_idx = rest_idx[val_num:(val_num+test_num)]
    return train_idx, val_idx,test_idx


train_idx,val_idx,test_idx = limit_data(labels)

label_counter:  {'Reinforcement_Learning': 0, 'Probabilistic_Methods': 0, 'Neural_Networks': 0, 'Case_Based': 0, 'Theory': 0, 'Genetic_Algorithms': 0, 'Rule_Learning': 0}
2708
[73, 80, 92, 98, 99, 103, 106, 107, 111, 112]


In [7]:
#set the mask
train_mask = np.zeros((N,),dtype=bool)
train_mask[train_idx] = True # with this way we can modify multiple numpy array elements at once  

val_mask = np.zeros((N,),dtype=bool)
val_mask[val_idx] = True

test_mask = np.zeros((N,),dtype=bool)
test_mask[test_idx] = True

In [8]:
train_mask

array([ True,  True,  True, ..., False, False, False])

<h3>Show Data Distribution</h3>

In [9]:
print("All Data Distribution: \n{}".format(Counter(labels)))

All Data Distribution: 
Counter({'Neural_Networks': 818, 'Probabilistic_Methods': 426, 'Genetic_Algorithms': 418, 'Theory': 351, 'Case_Based': 298, 'Reinforcement_Learning': 217, 'Rule_Learning': 180})


In [10]:
print("Training Data Distribution: \n{}".format(Counter([labels[i] for i in train_idx])))

Training Data Distribution: 
Counter({'Reinforcement_Learning': 20, 'Probabilistic_Methods': 20, 'Neural_Networks': 20, 'Case_Based': 20, 'Theory': 20, 'Genetic_Algorithms': 20, 'Rule_Learning': 20})


In [11]:
print("Validation Data Distribution: \n{}".format(Counter([labels[i] for i in val_idx])))

Validation Data Distribution: 
Counter({'Neural_Networks': 172, 'Genetic_Algorithms': 78, 'Probabilistic_Methods': 72, 'Theory': 63, 'Case_Based': 58, 'Reinforcement_Learning': 35, 'Rule_Learning': 22})


In [12]:
print("testing Data Distribution: \n{}".format(Counter([labels[i] for i in test_idx])))

testing Data Distribution: 
Counter({'Neural_Networks': 290, 'Probabilistic_Methods': 172, 'Genetic_Algorithms': 156, 'Theory': 123, 'Case_Based': 114, 'Reinforcement_Learning': 85, 'Rule_Learning': 60})


<h3>Convert the labels to one hot encoding</h3>

In [13]:
def encode_label(labels):
    label_encoder = LabelEncoder()#LabelEncoder is a utility class to help normalize labels such that they contain only values between 0 and n_classes-1. 
    print(label_encoder)
    labels = label_encoder.fit_transform(labels)#check the difference between of fit_tf() and tf()
    print(labels)
    labels = to_categorical(labels)
    print(labels)
    return labels, label_encoder.classes_
#print(labels)
labels_encoded, classes = encode_label(labels)

LabelEncoder()
[4 3 3 ... 2 6 2]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]]


In [14]:
classes

array(['Case_Based', 'Genetic_Algorithms', 'Neural_Networks',
       'Probabilistic_Methods', 'Reinforcement_Learning', 'Rule_Learning',
       'Theory'], dtype='<U22')

<h3>Build a graph on NetworkX using the obtained nodes and edges list</h3>

In [15]:
#build the graph
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list) #list of tuples

#obtain the adjacency matrix (A)
A = nx.adjacency_matrix(G)
print('Graph info: ', nx.info(G))

Graph info:  Name: 
Type: Graph
Number of nodes: 2708
Number of edges: 5278
Average degree:   3.8981


<h2> Building and Training Graph Convolutional Networks </h2>

In [16]:
# Parameters
channels = 16           # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 200            # Number of training epochs
es_patience = 10        # Patience for early stopping

# Preprocessing operations
A = GraphConv.preprocess(A).astype('f4')

# Model definition
X_in = Input(shape=(F, )) #F is number of features 1430+??
fltr_in = Input((N, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GraphConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])

dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GraphConv(num_classes,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in]) #num_classes =7

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
    log_dir='./Tensorboard_GCN_cora',
)
callback_GCN = [tbCallBack_GCN]

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 1433)         0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
graph_conv (GraphConv)          (None, 16)           22928       dropout[0][0]                    
                                                                 input_2[0][0]                

In [17]:
callback_GCN

In [18]:
# Train model
validation_data = ([X, A], labels_encoded, val_mask)
model.fit([X, A],
          labels_encoded,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[
              EarlyStopping(patience=es_patience,  restore_best_weights=True),
              tbCallBack_GCN
          ])

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 0.1167 - acc: 0.1500 - val_loss: 0.3657 - val_acc: 0.2540
Epoch 2/200
1/1 [==============================] - 0s 498ms/step - loss: 0.1095 - acc: 0.3071 - val_loss: 0.3563 - val_acc: 0.3120
Epoch 3/200
1/1 [==============================] - 0s 263ms/step - loss: 0.1032 - acc: 0.4071 - val_loss: 0.3468 - val_acc: 0.3600
Epoch 4/200
1/1 [==============================] - 0s 268ms/step - loss: 0.0976 - acc: 0.5000 - val_loss: 0.3378 - val_acc: 0.3860
Epoch 5/200
1/1 [==============================] - 0s 233ms/step - loss: 0.0932 - acc: 0.4929 - val_loss: 0.3285 - val_acc: 0.4140
Epoch 6/200
1/1 [==============================] - 0s 264ms/step - loss: 0.0890 - acc: 0.6071 - val_loss: 0.3200 - val_acc: 0.4440
Epoch 7/200
1/1 [==============================] - 0s 219ms/step - loss: 0.0855 - acc: 0.6929 - val_loss: 0.3118 - val_acc: 0.4960
Epoch 8/200
1/1 [==============================] - 0s 242ms/step - loss: 0.0808 - acc:

1/1 [==============================] - 0s 193ms/step - loss: 0.0358 - acc: 0.9429 - val_loss: 0.1600 - val_acc: 0.7860
Epoch 64/200
1/1 [==============================] - 0s 186ms/step - loss: 0.0334 - acc: 0.9786 - val_loss: 0.1598 - val_acc: 0.7820
Epoch 65/200
1/1 [==============================] - 0s 185ms/step - loss: 0.0336 - acc: 0.9571 - val_loss: 0.1592 - val_acc: 0.7840
Epoch 66/200
1/1 [==============================] - 0s 188ms/step - loss: 0.0369 - acc: 0.9571 - val_loss: 0.1593 - val_acc: 0.7800
Epoch 67/200
1/1 [==============================] - 0s 174ms/step - loss: 0.0334 - acc: 0.9643 - val_loss: 0.1591 - val_acc: 0.7820
Epoch 68/200
1/1 [==============================] - 0s 177ms/step - loss: 0.0358 - acc: 0.9500 - val_loss: 0.1586 - val_acc: 0.7860
Epoch 69/200
1/1 [==============================] - 0s 171ms/step - loss: 0.0332 - acc: 0.9857 - val_loss: 0.1581 - val_acc: 0.7900
Epoch 70/200
1/1 [==============================] - 0s 260ms/step - loss: 0.0336 - acc: 0

Epoch 125/200
1/1 [==============================] - 0s 185ms/step - loss: 0.0289 - acc: 0.9786 - val_loss: 0.1429 - val_acc: 0.7840
Epoch 126/200
1/1 [==============================] - 0s 180ms/step - loss: 0.0267 - acc: 0.9786 - val_loss: 0.1436 - val_acc: 0.7740
Epoch 127/200
1/1 [==============================] - 0s 190ms/step - loss: 0.0270 - acc: 0.9643 - val_loss: 0.1450 - val_acc: 0.7740
Epoch 128/200
1/1 [==============================] - 0s 187ms/step - loss: 0.0266 - acc: 0.9571 - val_loss: 0.1475 - val_acc: 0.7740
Epoch 129/200
1/1 [==============================] - 0s 185ms/step - loss: 0.0273 - acc: 0.9857 - val_loss: 0.1491 - val_acc: 0.7660
Epoch 130/200
1/1 [==============================] - 0s 178ms/step - loss: 0.0277 - acc: 0.9571 - val_loss: 0.1494 - val_acc: 0.7640
Epoch 131/200
1/1 [==============================] - 0s 183ms/step - loss: 0.0265 - acc: 0.9643 - val_loss: 0.1475 - val_acc: 0.7680
Epoch 132/200
1/1 [==============================] - 0s 183ms/step - 

In [19]:
N

2708

In [20]:
# Evaluate model
X_te = X[test_mask]
A_te = A[test_mask,:][:,test_mask]
y_te = labels_encoded[test_mask]

In [21]:
print(A_te.shape)

(1000, 1000)


In [22]:
# Evaluate model
X_te = X[test_mask]
A_te = A[test_mask,:][:,test_mask]
y_te = labels_encoded[test_mask]

y_pred = model.predict([X_te, A_te], batch_size=N)
report = classification_report(np.argmax(y_te,axis=1), np.argmax(y_pred,axis=1), target_names=classes)
print('GCN Classification Report: \n {}'.format(report))

ValueError: in user code:

    C:\Users\Sanka Mohottala\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\Sanka Mohottala\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Sanka Mohottala\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Sanka Mohottala\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Sanka Mohottala\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica

<h3> Get hidden layer representation for GCN </h3>

In [23]:
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict([X,A],batch_size=N)

#Get t-SNE Representation
#get the hidden layer representation after the first GCN layer
x_tsne = TSNE(n_components=2).fit_transform(activations[3]) 

In [ ]:
print(report)

In [ ]:
def plot_tSNE(labels_encoded,x_tsne):
    color_map = np.argmax(labels_encoded, axis=1)
    plt.figure(figsize=(10,10))
    for cl in range(num_classes):
        indices = np.where(color_map==cl)
        indices = indices[0]
        plt.scatter(x_tsne[indices,0], x_tsne[indices, 1], label=cl)
    plt.legend()
    plt.show()
    
plot_tSNE(labels_encoded,x_tsne)

<h2> Comparison to Fully-Connected Neural Networks </h2>

<h3>Building and Training FNN</h3>

In [ ]:
es_patience = 10
optimizer = Adam(lr=1e-2)
l2_reg = 5e-4
epochs = 200

#Compare with FNN
#Construct the model
model_fnn = Sequential()
model_fnn.add(Dense(
                    128,
                    input_dim=X.shape[1],
                    activation=tf.nn.relu,
                    kernel_regularizer=tf.keras.regularizers.l2(l2_reg))
             )
model_fnn.add(Dropout(0.5))
model_fnn.add(Dense(256, activation=tf.nn.relu))
model_fnn.add(Dropout(0.5))
model_fnn.add(Dense(num_classes, activation=tf.keras.activations.softmax))


model_fnn.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])


#define TensorBoard
tbCallBack_FNN = TensorBoard(
    log_dir='./Tensorboard_FNN_cora',
)

#Train model
validation_data_fnn = (X, labels_encoded, val_mask)
model_fnn.fit(
                X,labels_encoded,
                sample_weight=train_mask,
                epochs=epochs,
                batch_size=N,
                validation_data=validation_data_fnn,
                shuffle=False,
                callbacks=[
                  EarlyStopping(patience=es_patience,  restore_best_weights=True),
                  tbCallBack_FNN
          ])

In [ ]:
# Evaluate model
y_pred = model_fnn.predict(X_te)
report = classification_report(np.argmax(y_te,axis=1), np.argmax(y_pred,axis=1), target_names=classes)
print('FCNN Classification Report: \n {}'.format(report))

<h3>Get hidden layer representation for FNN</h3>

In [ ]:
layer_outputs = [layer.output for layer in model_fnn.layers] 
activation_model = Model(inputs=model_fnn.input, outputs=layer_outputs)
activations = activation_model.predict([X])

In [ ]:
x_tsne = TSNE(n_components=2).fit_transform(activations[3])
plot_tSNE(labels_encoded,x_tsne)

In [ ]:
### END OF NOTEBOOK ###